In [1]:
import gc,uuid
import pandas as pd
import numpy as np
import pyarrow as pa
import tensorflow as tf
from pyarrow import parquet as pq
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
import time_series_transform
from time_series_transform.time_series_transformer import Time_Series_Transformer
from time_series_transform.sequence_transfomer import *

In [2]:
CATEGORICAL_DIM = ['item_id','dept_id','cat_id','store_id','state_id']
WINDOW_SIZE = 10
TEST_RANGE = WINDOW_SIZE + 50
df = pd.read_csv('../Data/Train/sales_train_validation.csv')
df.index = df.id
df = df.drop('id',axis = 1)

In [3]:
testDate = list(map(lambda x: 'd_'+str(x),list(range(1913-TEST_RANGE,1914))))
trainDateDrop = []
test = df[CATEGORICAL_DIM+testDate]
train = df.drop(testDate+trainDateDrop,axis = 1)
train = train.sample(frac = 1)
SEQ_TRANSFORMER = []

In [4]:
fe = Time_Series_Transformer(train,['item_id','dept_id','cat_id','store_id','state_id'],seqTransformerList = SEQ_TRANSFORMER)
dept_id_class_num = fe.get_encoder_class('dept_id')
state_id_class_num = fe.get_encoder_class('state_id')
store_id_class_num = fe.get_encoder_class('store_id')
cat_id_class_num = fe.get_encoder_class('cat_id')


train_univariate,train_step = fe.get_tf_dataset(WINDOW_SIZE)
train_univariate = train_univariate.shuffle(buffer_size=10000).prefetch(2).repeat()

In [5]:
ge = Time_Series_Transformer(test,['item_id','dept_id','cat_id','store_id','state_id'],encodeDict= fe.encodeDict,seqTransformerList = SEQ_TRANSFORMER)

val_univariate,val_step = ge.get_tf_dataset(WINDOW_SIZE)
val_univariate = val_univariate.shuffle(buffer_size=10000).prefetch(2).repeat()

In [6]:
sells_input = tf.keras.layers.Input(shape=(WINDOW_SIZE,1),name = 'time_series')
cat_id_input= tf.keras.layers.Input(shape=1,name = 'cat_id')
store_id_input= tf.keras.layers.Input(shape=1,name = 'store_id')
dept_id_input = tf.keras.layers.Input(shape=1,name = 'dept_id')
state_id_input = tf.keras.layers.Input(shape=1,name = 'state_id')


dept_id = tf.keras.layers.Embedding(dept_id_class_num,2)(dept_id_input)
dept_id =tf.keras.layers.Flatten()(dept_id)

state_id = tf.keras.layers.Embedding(dept_id_class_num,2)(state_id_input)
state_id =tf.keras.layers.Flatten()(state_id)

cat_id = tf.keras.layers.Embedding(cat_id_class_num,2)(cat_id_input)
cat_id =tf.keras.layers.Flatten()(cat_id)

store_id = tf.keras.layers.Embedding(store_id_class_num,2)(store_id_input)
store_id =tf.keras.layers.Flatten()(store_id)

lstm = tf.keras.layers.Masking(np.nan)(sells_input)
lstm = tf.keras.layers.GRU(5)(lstm)

cate = tf.keras.layers.Concatenate()([lstm,cat_id,store_id,dept_id,state_id])

dense = tf.keras.layers.Dense(1,'relu')(lstm)
lstm_embed = tf.keras.models.Model({
    'time_series':sells_input,
    'cat_id':cat_id_input,
    'dept_id':dept_id_input,
    'state_id':state_id_input,
    'store_id':store_id_input
},dense)
lstm_embed.compile(optimizer='adam', loss='mse',metrics = ['mae','mse'])

In [7]:
tf.keras.utils.plot_model(lstm_embed,show_shapes=True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [8]:
EVALUATION_INTERVAL = len(list(fe.np_to_time_tensor_generator(WINDOW_SIZE)))
validation_steps =  10 #len(list(ge.np_to_time_tensor_generator(WINDOW_SIZE)))
EPOCHS = 10

In [9]:
lstm_embed.fit(
    train_univariate,
    epochs=EPOCHS,
    steps_per_epoch= EVALUATION_INTERVAL,
    validation_data=val_univariate, 
    validation_steps=validation_steps
    )

Train for 30490 steps, validate for 10 steps
Epoch 1/10
 1471/30490 [>.............................] - ETA: 26:57 - loss: 15.3834 - mae: 1.0535 - mse: 15.3834